<a href="https://colab.research.google.com/github/yzm10/yanxishe/blob/main/AI%E8%89%BA%E6%9C%AF%E9%89%B4%E8%B5%8F%E6%8C%91%E6%88%98%E8%B5%9B_%E7%9C%8B%E7%94%BB%E7%8C%9C%E4%BD%9C%E8%80%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI研习社 - 艺术鉴赏

预训练模型：SE-ResNeXt101_32x4d

epoch：24

score：65.625

~~made by yzm10~~

导入包

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import models,transforms,datasets
import time
import json


# 判断是否存在GPU设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using gpu: %s ' % torch.cuda.is_available())

Using gpu: True 


下载数据集

In [ ]:
# ! rm -rf ./train
# ! rm -rf ./test

In [ ]:
! wget http://qavs3w9z3.bkt.clouddn.com/Art.zip
! unzip Art.zip

读取csv，打标签

In [ ]:
import csv
csv_file = csv.reader(open('train.csv','r'))

AVATAR_STRANGER_PATH="./train/"

for name,label in csv_file:
  if name=='filename': continue
  
  image=os.path.join(AVATAR_STRANGER_PATH, name+'.jpg')
  print(image)
  
  # 分离目录和文件名
  # dirname, filename = os.path.split(image)
  # print(dirname, filename)
  
  # 改名
  new_file=os.path.join(AVATAR_STRANGER_PATH, label+'_'+name+'.jpg')
  print(new_file)

  os.rename(image, new_file)

训练集分类

In [4]:
! mkdir ./train/-0_Frida_Kahlo
! mkdir ./train/-1_Edgar_Degas
! mkdir ./train/-2_Pieter_Bruegel
! mkdir ./train/-3_Vincent_van_Gogh
! mkdir ./train/-4_Rembrandt
! mkdir ./train/-5_Henri_Rousseau
! mkdir ./train/-6_Henri_Matisse
! mkdir ./train/-7_Joan_Miro
! mkdir ./train/-8_Titian
! mkdir ./train/-9_Paul_Gauguin
! mkdir ./train/10_Pierre-Auguste_Renoir
! mkdir ./train/11_Marc_Chagall
! mkdir ./train/12_Raphael
! mkdir ./train/13_Leonardo_da_Vinci
! mkdir ./train/14_Amedeo_Modigliani
! mkdir ./train/15_Sandro_Botticelli
! mkdir ./train/16_Pablo_Picasso
! mkdir ./train/17_Rene_Magritte
! mkdir ./train/18_Vasiliy_Kandinskiy
! mkdir ./train/19_Salvador_Dali
! mkdir ./train/20_Michelangelo
! mkdir ./train/21_Mikhail_Vrubel
! mkdir ./train/22_Paul_Klee
! mkdir ./train/23_Camille_Pissarro
! mkdir ./train/24_Giotto_di_Bondone
! mkdir ./train/25_Gustave_Courbet
! mkdir ./train/26_Gustav_Klimt
! mkdir ./train/27_Henri_de_Toulouse-Lautrec
! mkdir ./train/28_Francisco_Goya
! mkdir ./train/29_Jan_van_Eyck
! mkdir ./train/30_Andrei_Rublev
! mkdir ./train/31_Andy_Warhol
! mkdir ./train/32_Alfred_Sisley
! mkdir ./train/33_Paul_Cezanne
! mkdir ./train/34_Diego_Velazquez
! mkdir ./train/35_Edouard_Manet
! mkdir ./train/36_Peter_Paul_Rubens
! mkdir ./train/37_Claude_Monet
! mkdir ./train/38_Kazimir_Malevich
! mkdir ./train/39_Hieronymus_Bosch
! mkdir ./train/40_Caravaggio
! mkdir ./train/41_Piet_Mondrian
! mkdir ./train/42_Diego_Rivera
! mkdir ./train/43_El_Greco
! mkdir ./train/44_William_Turner
! mkdir ./train/45_Georges_Seurat
! mkdir ./train/46_Jackson_Pollock
! mkdir ./train/47_Edvard_Munch
! mkdir ./train/48_Eugene_Delacroix

! mkdir ./test/test

! mv ./train/0_* ./train/-0_Frida_Kahlo
! mv ./train/1_* ./train/-1_Edgar_Degas
! mv ./train/2_* ./train/-2_Pieter_Bruegel
! mv ./train/3_* ./train/-3_Vincent_van_Gogh
! mv ./train/4_* ./train/-4_Rembrandt
! mv ./train/5_* ./train/-5_Henri_Rousseau
! mv ./train/6_* ./train/-6_Henri_Matisse
! mv ./train/7_* ./train/-7_Joan_Miro
! mv ./train/8_* ./train/-8_Titian
! mv ./train/9_* ./train/-9_Paul_Gauguin
! mv ./train/10_* ./train/10_Pierre-Auguste_Renoir
! mv ./train/11_* ./train/11_Marc_Chagall
! mv ./train/12_* ./train/12_Raphael
! mv ./train/13_* ./train/13_Leonardo_da_Vinci
! mv ./train/14_* ./train/14_Amedeo_Modigliani
! mv ./train/15_* ./train/15_Sandro_Botticelli
! mv ./train/16_* ./train/16_Pablo_Picasso
! mv ./train/17_* ./train/17_Rene_Magritte
! mv ./train/18_* ./train/18_Vasiliy_Kandinskiy
! mv ./train/19_* ./train/19_Salvador_Dali
! mv ./train/20_* ./train/20_Michelangelo
! mv ./train/21_* ./train/21_Mikhail_Vrubel
! mv ./train/22_* ./train/22_Paul_Klee
! mv ./train/23_* ./train/23_Camille_Pissarro
! mv ./train/24_* ./train/24_Giotto_di_Bondone
! mv ./train/25_* ./train/25_Gustave_Courbet
! mv ./train/26_* ./train/26_Gustav_Klimt
! mv ./train/27_* ./train/27_Henri_de_Toulouse-Lautrec
! mv ./train/28_* ./train/28_Francisco_Goya
! mv ./train/29_* ./train/29_Jan_van_Eyck
! mv ./train/30_* ./train/30_Andrei_Rublev
! mv ./train/31_* ./train/31_Andy_Warhol
! mv ./train/32_* ./train/32_Alfred_Sisley
! mv ./train/33_* ./train/33_Paul_Cezanne
! mv ./train/34_* ./train/34_Diego_Velazquez
! mv ./train/35_* ./train/35_Edouard_Manet
! mv ./train/36_* ./train/36_Peter_Paul_Rubens
! mv ./train/37_* ./train/37_Claude_Monet
! mv ./train/38_* ./train/38_Kazimir_Malevich
! mv ./train/39_* ./train/39_Hieronymus_Bosch
! mv ./train/40_* ./train/40_Caravaggio
! mv ./train/41_* ./train/41_Piet_Mondrian
! mv ./train/42_* ./train/42_Diego_Rivera
! mv ./train/43_* ./train/43_El_Greco
! mv ./train/44_* ./train/44_William_Turner
! mv ./train/45_* ./train/45_Georges_Seurat
! mv ./train/46_* ./train/46_Jackson_Pollock
! mv ./train/47_* ./train/47_Edvard_Munch
! mv ./train/48_* ./train/48_Eugene_Delacroix

! mv ./test/* ./test/test/

mv: cannot move './train/10_Pierre-Auguste_Renoir' to a subdirectory of itself, './train/10_Pierre-Auguste_Renoir/10_Pierre-Auguste_Renoir'
mv: cannot move './train/11_Marc_Chagall' to a subdirectory of itself, './train/11_Marc_Chagall/11_Marc_Chagall'
mv: cannot move './train/12_Raphael' to a subdirectory of itself, './train/12_Raphael/12_Raphael'
mv: cannot move './train/13_Leonardo_da_Vinci' to a subdirectory of itself, './train/13_Leonardo_da_Vinci/13_Leonardo_da_Vinci'
mv: cannot move './train/14_Amedeo_Modigliani' to a subdirectory of itself, './train/14_Amedeo_Modigliani/14_Amedeo_Modigliani'
mv: cannot move './train/15_Sandro_Botticelli' to a subdirectory of itself, './train/15_Sandro_Botticelli/15_Sandro_Botticelli'
mv: cannot move './train/16_Pablo_Picasso' to a subdirectory of itself, './train/16_Pablo_Picasso/16_Pablo_Picasso'
mv: cannot move './train/17_Rene_Magritte' to a subdirectory of itself, './train/17_Rene_Magritte/17_Rene_Magritte'
mv: cannot move './train/18_Vasil

数据处理

In [32]:
resnet_format = transforms.Compose([
  transforms.RandomHorizontalFlip(p=0.5),

  transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
  # transforms.Resize((224, 224)),
  transforms.ToTensor(), # ->(0,1)
  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # ->(-1,1)
])

data_dir = './'

dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), resnet_format)
         for x in ['train']}

tsets = {y: datasets.ImageFolder(os.path.join(data_dir, y), resnet_format)
        for y in ['test']}

dset_sizes = {x: len(dsets[x]) for x in ['train']}
dset_classes = dsets['train'].classes

# 通过下面代码可以查看 dsets 的一些属性

print(dsets['train'].classes)
print(dsets['train'].class_to_idx)
print(dsets['train'].imgs[:5])
print('dset_sizes: ', dset_sizes)

loader_train = torch.utils.data.DataLoader(dsets['train'], batch_size=64, shuffle=True, num_workers=6)

loader_test = torch.utils.data.DataLoader(tsets['test'],batch_size=8,shuffle=False,num_workers=6)

['-0_Frida_Kahlo', '-1_Edgar_Degas', '-2_Pieter_Bruegel', '-3_Vincent_van_Gogh', '-4_Rembrandt', '-5_Henri_Rousseau', '-6_Henri_Matisse', '-7_Joan_Miro', '-8_Titian', '-9_Paul_Gauguin', '10_Pierre-Auguste_Renoir', '11_Marc_Chagall', '12_Raphael', '13_Leonardo_da_Vinci', '14_Amedeo_Modigliani', '15_Sandro_Botticelli', '16_Pablo_Picasso', '17_Rene_Magritte', '18_Vasiliy_Kandinskiy', '19_Salvador_Dali', '20_Michelangelo', '21_Mikhail_Vrubel', '22_Paul_Klee', '23_Camille_Pissarro', '24_Giotto_di_Bondone', '25_Gustave_Courbet', '26_Gustav_Klimt', '27_Henri_de_Toulouse-Lautrec', '28_Francisco_Goya', '29_Jan_van_Eyck', '30_Andrei_Rublev', '31_Andy_Warhol', '32_Alfred_Sisley', '33_Paul_Cezanne', '34_Diego_Velazquez', '35_Edouard_Manet', '36_Peter_Paul_Rubens', '37_Claude_Monet', '38_Kazimir_Malevich', '39_Hieronymus_Bosch', '40_Caravaggio', '41_Piet_Mondrian', '42_Diego_Rivera', '43_El_Greco', '44_William_Turner', '45_Georges_Seurat', '46_Jackson_Pollock', '47_Edvard_Munch', '48_Eugene_Delacro

建模

In [11]:
# se_resnext101_32x4d
! pip install pretrainedmodels
import pretrainedmodels

     |████████████████████████████████| 61kB 2.8MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp36-none-any.whl size=60964 sha256=b899a89fab1ea087f9ff3d20437a12219d538085c9c8cb2da2ee89b5b290da41
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
Successfully built pretrainedmodels


In [12]:
model_1 = pretrainedmodels.se_resnext101_32x4d(num_classes=1000, pretrained='imagenet')

print('1****************')
print(model_1)

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext101_32x4d-3b2fe3d8.pth" to /root/.cache/torch/hub/checkpoints/se_resnext101_32x4d-3b2fe3d8.pth



1****************
SENet(
  (layer0): Sequential(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  )
  (layer1): Sequential(
    (0): SEResNeXtBottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (se_modul

fine-tune

In [13]:
# for param in model_1.parameters():
#   param.requires_grad = False

model_1.last_linear = nn.Sequential(
    nn.Linear(2048, 49)
)

model_1 = model_1.to(device)

print('1****************')
print(model_1.last_linear)

1****************
Sequential(
  (0): Linear(in_features=2048, out_features=49, bias=True)
)


训练

In [33]:
'''
第一步：创建损失函数和优化器

损失函数 NLLLoss() 的 输入 是一个对数概率向量和一个目标标签. 
它不会为我们计算对数概率，适合最后一层是log_softmax()的网络. 
'''
# criterion = nn.NLLLoss()
# 二分类：NLLLoss（负对数似然损失函数）
criterion = nn.CrossEntropyLoss()
# 多分类：CrossEntropyLoss（交叉熵损失函数）

# 学习率 Adam最优3e-4
lr = 0.0003

# 随机梯度下降
# optimizer_vgg = torch.optim.SGD(model_vgg_new.classifier[6].parameters(),lr = lr)
# Adam
optimizer_1 = torch.optim.Adam(model_1.last_linear.parameters(),lr = lr)

def smooth_one_hot(true_labels: torch.Tensor, classes: int, smoothing=0.0):
    """
    if smoothing == 0, it's one-hot method
    if 0 < smoothing < 1, it's smooth method

    """
    assert 0 <= smoothing < 1
    confidence = 1.0 - smoothing
    label_shape = torch.Size((true_labels.size(0), classes))
    with torch.no_grad():
        true_dist = torch.empty(size=label_shape, device=true_labels.device)
        true_dist.fill_(smoothing / (classes - 1))
        true_dist.scatter_(1, true_labels.data.unsqueeze(1), confidence)
    return true_dist

'''
第二步：训练模型
'''

def train_model(model,dataloader,size,epochs=1,optimizer=None):
    model.train()
    
    for epoch in range(epochs):
        running_loss = 0.0
        running_corrects = 0
        count = 0
        for i, (inputs, classes) in enumerate(dataloader):
            inputs = inputs.to(device)
            classes = classes.to(device)

            outputs = model(inputs)

            # smooth_label = smooth_one_hot(classes, classes=6, smoothing=0.1)
            loss = criterion(outputs, classes)           
            
            optimizer = optimizer
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if i % 10 == 0:   
              print('Epoch: %d Minibatch: %5d loss: %.3f' %(epoch + 1, i + 1, loss.item()))

            _,preds = torch.max(outputs.data,1)
            # statistics
            running_loss += loss.data.item()
            running_corrects += torch.sum(preds == classes.data)
            count += len(inputs)
            print('Training: No. ', count, ' process ... total: ', size)
        epoch_loss = running_loss / size
        epoch_acc = running_corrects.data.item() / size
        print('Loss: {:.4f} Acc: {:.4f}'.format(
                     epoch_loss, epoch_acc))
        
        
# 模型训练
print('1****************')
train_model(model_1,loader_train,size=dset_sizes['train'], epochs=24, optimizer=optimizer_1)

1****************
Epoch: 1 Minibatch:     1 loss: 0.326
Training: No.  64  process ... total:  7227
Training: No.  128  process ... total:  7227
Training: No.  192  process ... total:  7227
Training: No.  256  process ... total:  7227
Training: No.  320  process ... total:  7227
Training: No.  384  process ... total:  7227
Training: No.  448  process ... total:  7227
Training: No.  512  process ... total:  7227
Training: No.  576  process ... total:  7227
Training: No.  640  process ... total:  7227
Epoch: 1 Minibatch:    11 loss: 0.211
Training: No.  704  process ... total:  7227
Training: No.  768  process ... total:  7227
Training: No.  832  process ... total:  7227
Training: No.  896  process ... total:  7227
Training: No.  960  process ... total:  7227
Training: No.  1024  process ... total:  7227
Training: No.  1088  process ... total:  7227
Training: No.  1152  process ... total:  7227
Training: No.  1216  process ... total:  7227
Training: No.  1280  process ... total:  7227
Ep

输出结果

In [34]:
import re

def answer(model):
  # 得到测试图片编号
  string = tsets['test'].imgs[0][0]
  num = re.sub("\D", "", string)

  result_pro = []
  result_pre = []
  # 将测试数据集（test）输入到网络中，得到识别结果
  for item,lable in loader_test:
    item = item.to(device)
    ll =  model(item)
    pro, pre = torch.max(ll.data,1)

    result_pro += pro
    result_pre += pre

  # 结果排序
  result_end =list()

  for i in range(800):
    string = tsets['test'].imgs[i][0]
    num = re.sub("\D", "", string)
    result_end.append((num, result_pre[i].item(), result_pro[i].item()))

  result_sort = sorted(result_end,key=lambda x:int(x[0]))
  return result_sort

result_1 = answer(model_1)

print('1****************')
print(result_1)

import csv
# 写入文件

f = open('out_file.csv','w')
writer = csv.writer(f)
for id, res, _ in result_1:

  ans = id, res
  writer.writerow(ans)

f.close()

1****************
[('0', 44, -3.2966418266296387), ('1', 3, -4.870187759399414), ('2', 15, -5.813105583190918), ('3', 9, -5.99786901473999), ('4', 9, -2.535879611968994), ('5', 16, -3.811992883682251), ('6', 5, -5.864582538604736), ('7', 24, -6.485613822937012), ('8', 3, -6.226531028747559), ('9', 10, -1.7814123630523682), ('10', 8, -5.5818867683410645), ('11', 3, -3.3076798915863037), ('12', 10, -2.256056070327759), ('13', 1, -3.611907482147217), ('14', 18, -2.838205575942993), ('15', 9, -7.295907497406006), ('16', 1, -7.358201503753662), ('17', 1, -8.871493339538574), ('18', 16, -5.440230369567871), ('19', 1, -4.129091739654541), ('20', 36, -7.898248672485352), ('21', 15, -8.858686447143555), ('22', 17, -2.9175167083740234), ('23', 14, -3.730212926864624), ('24', 35, -6.656628131866455), ('25', 8, -7.416707515716553), ('26', 9, -4.200602054595947), ('27', 30, -7.402402877807617), ('28', 11, -4.929100513458252), ('29', 3, -3.688328504562378), ('30', 8, -5.9021735191345215), ('31', 21,